<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Weekly newsletter based on issue/PR activity
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/GitHub_weekly_newsletter_based_on_issue_PR_activity.ipynb.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #tool #naas_drivers #naas #scheduler #asset #snippet #automation #ai #newsletter

**Author:** [Suhas B](https://www.linkedin.com/in/suhasbrao/)

The notebook allows you to create a weekly newsletter based on issue/PR activity for a GitHub repository.

## Input

### Import library

In [ ]:
from naas_drivers import github
import naas
import getpass
import markdown2
from IPython.core.display import display, HTML

### Variables

Before running the below cell ensure you have added a GitHub access token for repository using naas 
by running **naas.secret.add(name="API_NAME", secret="API_KEY")** and delete that cell.

You can find more info [here](https://docs.naas.ai/features/secret#add-or-edit-secret)

In [ ]:
# naas.secret.list()
GITHUB_TOKEN = naas.secret.get(name="GITHUB_TOKEN")
repo_url = "https://github.com/jupyter-naas/awesome-notebooks"
repo_name = "awesome-notebooks"

In [ ]:
email_receivers = ["raos04567@gmail.com"] #list of email_receivers

no_activities_to_show = 5

# EMAIL_FROM = None
EMAIL_SUBJECT = "✉️ Weekly Repository activity "

content_image_src = "https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"

# Markdown template created on your local env
EMAIL_CONTENT_MD = "email_content.md"

### Schedule your notebook

Scheduling notebook to run At 12:00 on Sunday. You can modify it if you want 

In [ ]:
# Schedule your notebook 
naas.scheduler.add(cron="0 12 * * 0")

#-> Uncomment the line below to remove your scheduler
# naas.scheduler.delete()

## Model

### Get data from GitHub about issues and PR 

In [ ]:
df_issues = github.connect(GITHUB_TOKEN).repos.get_issues(repo_url)
df_pulls = github.connect(GITHUB_TOKEN).repos.get_pulls(repo_url)


## Create a Content Markdown file
This is later to converted to a HTML content for the mail. Modify these contents based on your needs.

In [ ]:
%%writefile $EMAIL_CONTENT_MD

<img width="10%" alt="Naas" src=img_src
style="width:640px; height:360px;"/>

Hello there,

We are back again with this weeks activity memo 📝 on GitHub repository **repo_name**

### In this week's Issues 📰:

- [#IssueNumber0-IssueName0](IssueURL0) last updated on **IssueUpdatedDate0** at **IssueUpdatedTime0**
- [#IssueNumber1-IssueName1](IssueURL1) last updated on **IssueUpdatedDate1** at **IssueUpdatedTime1**
- [#IssueNumber2-IssueName2](IssueURL2) last updated on **IssueUpdatedDate2** at **IssueUpdatedTime2**
- [#IssueNumber3-IssueName3](IssueURL3) last updated on **IssueUpdatedDate3** at **IssueUpdatedTime3**
- [#IssueNumber4-IssueName4](IssueURL4) last updated on **IssueUpdatedDate4** at **IssueUpdatedTime4**

### Pull Requests ⭐ Highlights,
    
- [PRnumber0 - PRName0](PRURL0) with assignees **PRassignees0** and requested reviewers **PRreviewers0**
- [PRnumber1 - PRName1](PRURL1) with assignees **PRassignees1** and requested reviewers **PRreviewers1**
- [PRnumber2 - PRName2](PRURL2) with assignees **PRassignees2** and requested reviewers **PRreviewers2**
- [PRnumber3 - PRName3](PRURL3) with assignees **PRassignees3** and requested reviewers **PRreviewers3**
- [PRnumber4 - PRName4](PRURL4) with assignees **PRassignees4** and requested reviewers **PRreviewers4**

Have a nice day 😆.
<br>


<div><strong>repo_name</strong></div>
<div>Open source lover | <a href="http://www.naas.ai/" target="_blank">Naas</a></div>
<div><small>This is an automated email from my Naas account</small></div>

In [ ]:
def replace_values(md):
    post = md.replace("repo_name", str(repo_name))
    
    post = post.replace("img_src", content_image_src)
    
    for index in range(no_activities_to_show):
        str_index = str(index)
        
        issue_number = "IssueNumber" + str_index
        issue_name = "IssueName" + str_index
        issue_updated_date = "IssueUpdatedDate" + str_index
        issue_updated_time = "IssueUpdatedTime" + str_index
        issue_url = "IssueURL" + str_index
        
        pr_number = "PRnumber" + str_index
        pr_name = "PRName" + str_index
        pr_url = "PRURL" + str_index
        pr_assignees = "PRassignees" + str_index
        pr_reviewers = "PRreviewers" + str_index
        
        # Chaning the issues details
        post = post.replace(issue_number, str(df_issues["issue_number"][index]))
        post = post.replace(issue_name, str(df_issues["issue_title"][index]))
        post = post.replace(issue_updated_date, str(df_issues["last_updated_date"][index]))
        post = post.replace(issue_updated_time, str(df_issues["last_updated_time"][index]))
        post = post.replace(issue_url, str(df_issues["link_to_the_issue"][index]))
        
        #changing the PR details
        post = post.replace(pr_number, str(df_pulls["PR_number"][index]))
        post = post.replace(pr_name, str(df_pulls["Title"][index]))
        post = post.replace(pr_url, str(df_pulls["issue_url"][index]))
        post = post.replace(pr_assignees, str(df_pulls["assignees"][index]))
        post = post.replace(pr_reviewers, str(df_pulls["requested_reviewers"][index]))
        

    return post

In [ ]:
content = open(EMAIL_CONTENT_MD, "r").read()
md = markdown2.markdown(content)
email_content = replace_values(md)
display(HTML(email_content))

## Output

In [ ]:
print("Nb issues:", len(df_issues))
df_issues.head(5)

In [ ]:
print("Nb PRs:", len(df_pulls))
df_pulls.head(5)
# df_pulls["PR_activity"][1]

### Send mail to receivers

In [ ]:
for receiver in email_receivers:
    naas.notification.send(email_to=receiver,
                           subject=EMAIL_SUBJECT,
                           html=email_content)